In [ ]:
df = pd.read_excel('1.xlsx')

#检查缺失值并删除包含缺失值的行
df = df.dropna()

#分离特征和标签
X= df.drop(columns=['diagnosis'])
y= df['diagnosis']

#将分类变量转换为独热编码
categorical_cols =['1', '2', '3', '4']
encoder = OneHotEncoder(drop='first', sparse_output=False)
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_cols]), columns=encoder.get_feature_names_out(categorical_cols))
X = pd.concat([X.drop(columns=categorical_cols), X_encoded], axis=1)

# 标准化数值特征
val_cols =['5', '6', '7', '8', '9', '10', '11']
scaler=StandardScaler()
X[val_cols]=scaler.fit_transform(X[val_cols])

#将数据转换为 float32 类型
X=X.astype(np.float32)

In [ ]:
# 分割数据集
# 分割数据集为训练集和测试集，测试集占30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# 定义每个模型的参数网格
param_grids = {
    'log_reg': {
        'C': [0.1, 1, 10],
        'penalty': ['l2'],
        'solver': ['lbfgs', 'saga']
    },
    'rf': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10]
    },
    'svm': {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf'],
        'gamma': ['scale', 'auto']
    },
    'xgb': {
        'learning_rate': [0.01, 0.1, 0.3],
        'max_depth': [3, 5, 7],
        'n_estimators': [100, 200],
        'subsample': [0.8, 1.0]
    },
    'lgb': {
        'learning_rate': [0.01, 0.1, 0.3],
        'num_leaves': [31, 50, 100],
        'n_estimators': [100, 200],
        'boosting_type': ['gbdt', 'dart']
    },
    'mlp': {
        'hidden_layer_sizes': [(50,), (100,), (100, 50)],
        'learning_rate_init': [0.001, 0.01],
        'max_iter': [200, 300]
    }
}

# 创建模型实例
models = {
    'log_reg': LogisticRegression(random_state=42),
    'rf': RandomForestClassifier(random_state=42),
    'svm': SVC(probability=True, random_state=42),  # 注意：probability=True
    'xgb': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    'lgb': LGBMClassifier(random_state=42),
    'mlp': MLPClassifier(random_state=42)
}

# 存储最佳模型
best_models = {}

# 遍历每个模型及其对应的参数网格进行网格搜索
for model_name in models:
    print(f"Performing grid search for {model_name}...")
    
    grid_search = GridSearchCV(
        estimator=models[model_name],  # 修正拼写
        param_grid=param_grids[model_name],  # 修正拼写
        scoring='roc_auc',
        cv=5,
        n_jobs=1,
        verbose=1
    )
    
    # 进行网格搜索并寻找最佳参数
    grid_search.fit(X_train, y_train)
    
    # 打印最佳参数
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    print(f"Best AUC score: {grid_search.best_score_}\n")
    
    # 保存最佳模型
    best_models[model_name] = grid_search.best_estimator_

#训练集ROC
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# 计算AUC的95%置信区间
def compute_ci(y_true, y_pred, n_bootstraps=1000, alpha=0.95):
    bootstrapped_scores = []
    rng = np.random.RandomState(seed=123)

    # 将y_true转换为numpy数组
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    for i in range(n_bootstraps):
        # 生成带放回的随机采样
        indices = rng.randint(0, len(y_pred), len(y_pred))

        # 确保采样后的y_true至少有两个唯一值，否则跳过
        if len(np.unique(y_true[indices])) < 2:
            continue

        # 计算ROC曲线并得到AUC
        score = auc(*roc_curve(y_true[indices], y_pred[indices])[:2])
        bootstrapped_scores.append(score)

    sorted_scores = np.sort(bootstrapped_scores)
    
    # 计算下界和上界
    lower_bound = np.percentile(sorted_scores, (1 - alpha) / 2 * 100)
    upper_bound = np.percentile(sorted_scores, (alpha + (1 - alpha) / 2) * 100)
    
    return lower_bound, upper_bound

# 绘制ROC曲线的函数
def plot_roc_curve(model, X, y, label, is_lgb=False):
    if is_lgb:
        try:
            y_pred = model.predict_proba(X, num_iteration=model.best_iteration_)[:, 1]
        except AttributeError:
            y_pred = model.predict_proba(X)[:, 1]
    else:
        y_pred = model.predict_proba(X)[:, 1]

    # 计算ROC曲线和AUC
    fpr, tpr, _ = roc_curve(y, y_pred)
    roc_auc = auc(fpr, tpr)

    # 计算AUC的95%置信区间
    ci_lower, ci_upper = compute_ci(y, y_pred)

    # 绘制ROC曲线并显示AUC和置信区间，格式调整
    plt.plot(fpr, tpr, label=f'{label} (AUC = {roc_auc:.3f} 95% CI [{ci_lower:.3f}-{ci_upper:.3f}])')

# 绘制不同模型的ROC曲线
plt.figure(figsize=(8, 6))

# 逻辑回归
plot_roc_curve(best_models['log_reg'], X_train, y_train, 'LR')

# 随机森林
plot_roc_curve(best_models['rf'], X_train, y_train, 'RF')

# 多层感知器(MLP)
plot_roc_curve(best_models['mlp'], X_train, y_train, 'MLP')

# SVM
plot_roc_curve(best_models['svm'], X_train, y_train, 'SVM')

# XGBoost
plot_roc_curve(best_models['xgb'], X_train, y_train, 'XGBoost')

# LightGBM
plot_roc_curve(best_models['lgb'], X_train, y_train, 'GBM', is_lgb=True)

# 添加对角线(随机分类器的参考线)
plt.plot([0, 1], [0, 1], 'k--', label='Random classifier (Diagonal)')

# 设置图例和标签
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves (Training set)')
plt.legend(loc='lower right')

# 保存图像
plt.savefig('roc1.png')

# 显示图像
plt.show()

#测试集ROC
import numpy as np
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# 计算AUC的95%置信区间
def compute_ci(y_true, y_pred, n_bootstraps=1000, alpha=0.95):
    bootstrapped_scores = []
    rng = np.random.RandomState(seed=123)

    # 将y_true转换为numpy数组
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    for i in range(n_bootstraps):
        # 生成带放回的随机采样
        indices = rng.randint(0, len(y_pred), len(y_pred))

        # 确保采样后的y_true至少有两个唯一值，否则跳过
        if len(np.unique(y_true[indices])) < 2:
            continue

        # 计算ROC曲线并得到AUC
        score = auc(*roc_curve(y_true[indices], y_pred[indices])[:2])
        bootstrapped_scores.append(score)

    sorted_scores = np.sort(bootstrapped_scores)
    
    # 计算下界和上界
    lower_bound = np.percentile(sorted_scores, (1 - alpha) / 2 * 100)
    upper_bound = np.percentile(sorted_scores, (alpha + (1 - alpha) / 2) * 100)
    
    return lower_bound, upper_bound

# 绘制ROC曲线的函数
def plot_roc_curve(model, X, y, label, is_lgb=False):
    if is_lgb:
        try:
            y_pred = model.predict_proba(X, num_iteration=model.best_iteration_)[:, 1]
        except AttributeError:
            y_pred = model.predict_proba(X)[:, 1]
    else:
        y_pred = model.predict_proba(X)[:, 1]

    # 计算ROC曲线和AUC
    fpr, tpr, _ = roc_curve(y, y_pred)
    roc_auc = auc(fpr, tpr)

    # 计算AUC的95%置信区间
    ci_lower, ci_upper = compute_ci(y, y_pred)

    # 绘制ROC曲线并显示AUC和置信区间，格式调整
    plt.plot(fpr, tpr, label=f'{label} (AUC = {roc_auc:.3f} 95% CI [{ci_lower:.3f}-{ci_upper:.3f}])')

# 绘制不同模型的ROC曲线
plt.figure(figsize=(8, 6))

# 逻辑回归
plot_roc_curve(best_models['log_reg'], X_test, y_test, 'LR')

# 随机森林
plot_roc_curve(best_models['rf'], X_test, y_test, 'RF')

# 多层感知器(MLP)
plot_roc_curve(best_models['mlp'], X_test, y_test, 'MLP')

# SVM
plot_roc_curve(best_models['svm'], X_test, y_test, 'SVM')

# XGBoost
plot_roc_curve(best_models['xgb'], X_test, y_test, 'XGBoost')

# LightGBM
plot_roc_curve(best_models['lgb'], X_test, y_test, 'GBM', is_lgb=True)

# 添加对角线(随机分类器的参考线)
plt.plot([0, 1], [0, 1], 'k--', label='Random classifier (Diagonal)')

# 设置图例和标签
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves (Test set)')
plt.legend(loc='lower right')

# 保存图像
plt.savefig('roc2png')

# 显示图像
plt.show()

# 加载外部验证集
df_external = pd.read_csv('external.csv')

# 检查缺失值并删除包含缺失值的行
df_external = df_external.dropna()


# 对分类变量进行独热编码
X_external_encoded = pd.DataFrame(encoder.transform(X_external[categorical_cols]), columns=encoder.get_feature_names_out(categorical_cols))
X_external = pd.concat([X_external.drop(columns=categorical_cols), X_external_encoded], axis=1)

# 对数值特征进行标准化
X_external[val_cols] = scaler.transform(X_external[val_cols])

# 确保数据类型一致
X_external = X_external.astype(np.float32)

# 绘制外部验证集的ROC曲线
plt.figure(figsize=(8, 6))

# 使用最佳参数的模型绘制外部验证集的ROC曲线

# 逻辑回归
plot_roc_curve(best_models['log_reg'], X_external, y_external, 'Logistic Regression')

# 随机森林
plot_roc_curve(best_models['rf'], X_external, y_external, 'Random Forest')

# 多层感知器(MLP)
plot_roc_curve(best_models['mlp'], X_external, y_external, 'MLP')

# SVM
plot_roc_curve(best_models['svm'], X_external, y_external, 'SVM')

# XGBoost
plot_roc_curve(best_models['xgb'], X_external, y_external, 'XGBoost')

# LightGBM
plot_roc_curve(best_models['lgb'], X_external, y_external, 'GBM', is_lgb=True)

# 添加对角线(随机分类器的参考线)
plt.plot([0, 1], [0, 1], 'k--', label='Random classifier (Diagonal)')

# 设置图例和标签
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves (External Validation Set)')
plt.legend(loc='lower right')

# 保存图像
plt.savefig('roc_curves_external.png')

# 显示图像
plt.show()

#训练集校准曲线
import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
from sklearn.utils import resample
from sklearn.calibration import CalibratedClassifierCV

def brier_score_confidence_interval(y_true, y_pred_prob, n_bootstraps=1000, alpha=0.05):
    brier_scores = []
    
    for _ in range(n_bootstraps):
        indices = resample(np.arange(len(y_true)), n_samples=len(y_true), replace=True)
        y_true_sample = np.array(y_true)[indices]
        y_pred_prob_sample = np.array(y_pred_prob)[indices]
        
        brier_score = np.mean((y_pred_prob_sample - y_true_sample) ** 2)
        brier_scores.append(brier_score)
    
    brier_mean = np.mean(brier_scores)
    brier_lower = np.percentile(brier_scores, (alpha / 2) * 100)
    brier_upper = np.percentile(brier_scores, (1 - alpha / 2) * 100)
    
    return brier_mean, brier_lower, brier_upper

# 绘制校准曲线函数（多模型）
def plot_calibration_curves(models, model_names, X, y, dataset_name):
    plt.figure(figsize=(8, 6))
    plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectly Calibrated')

    for model, name in zip(models, model_names):
        y_pred_prob = model.predict_proba(X)[:, 1]
        
        prob_true, prob_pred = calibration_curve(y, y_pred_prob, n_bins=10)
        
        brier_mean, brier_lower, brier_upper = brier_score_confidence_interval(y, y_pred_prob)
        
        plt.plot(prob_pred, prob_true, marker='o', linewidth=1, label=f'{name} ({brier_mean:.3f} 95%CI [{brier_lower:.3f}-{brier_upper:.3f}])')

    plt.title(f'Calibration Curve ({dataset_name})')
    plt.xlabel('Mean predicted value')
    plt.ylabel('Fraction of positives')
    plt.legend(loc='best')
    plt.savefig(f'calibration_curve_{dataset_name.lower()}.png')
    plt.show()

# 加载训练好的模型
xgb_model = best_models['xgb']
logistic_model = best_models['log_reg']
rf_model = best_models['rf']
svm_model = best_models['svm']
mlp_model = best_models['mlp']
lgb_model = best_models['lgb']

# 使用 CalibratedClassifierCV 对所有模型进行校准
models = [
    CalibratedClassifierCV(xgb_model, cv=5, method='sigmoid'),
    CalibratedClassifierCV(logistic_model, cv=5, method='sigmoid'),
    CalibratedClassifierCV(rf_model, cv=5, method='sigmoid'),
    CalibratedClassifierCV(svm_model, cv=5, method='sigmoid'),
    CalibratedClassifierCV(mlp_model, cv=5, method='sigmoid'),
    CalibratedClassifierCV(lgb_model, cv=5, method='sigmoid')
]

# 训练校准后的模型
for model in models:
    model.fit(X_train, y_train)

# 模型名称
model_names = ['XGBoost', 'Logistic Regression', 'Random Forest', 'SVM', 'MLP', 'LightGBM']

# 绘制校准曲线并计算 Brier 分数
plot_calibration_curves(models, model_names, X_train, y_train, 'Training Set')

#测试集校准曲线
import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
from sklearn.utils import resample
from sklearn.calibration import CalibratedClassifierCV

def brier_score_confidence_interval(y_true, y_pred_prob, n_bootstraps=1000, alpha=0.05):
    brier_scores = []
    
    for _ in range(n_bootstraps):
        indices = resample(np.arange(len(y_true)), n_samples=len(y_true), replace=True)
        y_true_sample = np.array(y_true)[indices]
        y_pred_prob_sample = np.array(y_pred_prob)[indices]
        
        brier_score = np.mean((y_pred_prob_sample - y_true_sample) ** 2)
        brier_scores.append(brier_score)
    
    brier_mean = np.mean(brier_scores)
    brier_lower = np.percentile(brier_scores, (alpha / 2) * 100)
    brier_upper = np.percentile(brier_scores, (1 - alpha / 2) * 100)
    
    return brier_mean, brier_lower, brier_upper

# 绘制校准曲线函数（多模型）
def plot_calibration_curves(models, model_names, X, y, dataset_name):
    plt.figure(figsize=(8, 6))
    plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectly Calibrated')

    for model, name in zip(models, model_names):
        y_pred_prob = model.predict_proba(X)[:, 1]
        
        prob_true, prob_pred = calibration_curve(y, y_pred_prob, n_bins=10)
        
        brier_mean, brier_lower, brier_upper = brier_score_confidence_interval(y, y_pred_prob)
        
        plt.plot(prob_pred, prob_true, marker='o', linewidth=1, label=f'{name} ({brier_mean:.3f} 95%CI [{brier_lower:.3f}-{brier_upper:.3f}])')

    plt.title(f'Calibration Curve ({dataset_name})')
    plt.xlabel('Mean predicted value')
    plt.ylabel('Fraction of positives')
    plt.legend(loc='best')
    plt.savefig(f'calibration_curve_17变量男性{dataset_name.lower()}.png')
    plt.show()

# 加载训练好的模型
xgb_model = best_models['xgb']
logistic_model = best_models['log_reg']
rf_model = best_models['rf']
svm_model = best_models['svm']
mlp_model = best_models['mlp']
lgb_model = best_models['lgb']

# 使用 CalibratedClassifierCV 对所有模型进行校准
models = [
    CalibratedClassifierCV(xgb_model, cv=5, method='sigmoid'),
    CalibratedClassifierCV(logistic_model, cv=5, method='sigmoid'),
    CalibratedClassifierCV(rf_model, cv=5, method='sigmoid'),
    CalibratedClassifierCV(svm_model, cv=5, method='sigmoid'),
    CalibratedClassifierCV(mlp_model, cv=5, method='sigmoid'),
    CalibratedClassifierCV(lgb_model, cv=5, method='sigmoid')
]
# 训练校准后的模型
for model in models:
    model.fit(X_train, y_train)

# 模型名称
model_names = ['XGBoost', 'Logistic Regression', 'Random Forest', 'SVM', 'MLP', 'LightGBM']

# 绘制校准曲线并计算 Brier 分数
plot_calibration_curves(models, model_names, X_test, y_test, 'test Set')

import numpy as np
import matplotlib.pyplot as plt
from sklearn.calibration import calibration_curve
from sklearn.metrics import brier_score_loss
from sklearn.utils import resample
from sklearn.calibration import CalibratedClassifierCV

# 计算Brier分数的置信区间
def brier_score_confidence_interval(y_true, y_pred_prob, n_bootstraps=1000, alpha=0.05):
    brier_scores = []
    
    for _ in range(n_bootstraps):
        indices = resample(np.arange(len(y_true)), n_samples=len(y_true), replace=True)
        y_true_sample = np.array(y_true)[indices]
        y_pred_prob_sample = np.array(y_pred_prob)[indices]
        
        # 计算Brier分数
        brier_score = np.mean((y_pred_prob_sample - y_true_sample) ** 2)
        brier_scores.append(brier_score)
    
    # 计算均值和置信区间
    brier_mean = np.mean(brier_scores)
    brier_lower = np.percentile(brier_scores, (alpha / 2) * 100)
    brier_upper = np.percentile(brier_scores, (1 - alpha / 2) * 100)
    
    return brier_mean, brier_lower, brier_upper

# 绘制校准曲线函数
# 绘制校准曲线函数
def plot_calibration_curves(models, model_names, X, y, dataset_name):
    plt.figure(figsize=(8, 6))
    plt.plot([0, 1], [0, 1], linestyle='--', label='Perfectly Calibrated')

    for model, name in zip(models, model_names):
        # 预测概率
        y_pred_prob = model.predict_proba(X)[:, 1]
        
        # 计算校准曲线
        prob_true, prob_pred = calibration_curve(y, y_pred_prob, n_bins=10)
        
        # 计算 Brier 分数和置信区间
        brier_mean, brier_lower, brier_upper = brier_score_confidence_interval(y, y_pred_prob)
        
        # 绘制校准曲线
        plt.plot(prob_pred, prob_true, marker='o', linewidth=1, 
                 label=f'{name} [Brier={brier_mean:.3f} 95%CI ({brier_lower:.3f}-{brier_upper:.3f})]')

    plt.title(f'Calibration Curve ({dataset_name})')
    plt.xlabel('Mean predicted value')
    plt.ylabel('Fraction of positives')
    
    # 将图例固定在右下角
    plt.legend(loc='lower right')
    
    # 保存图像
    plt.savefig(f'calibration_curve_{dataset_name.lower()}.png')
    plt.show()


# 训练和校准模型
def train_and_calibrate_models(best_models, X_train, y_train):
    # 提取模型
    xgb_model = best_models['xgb']
    logistic_model = best_models['log_reg']
    rf_model = best_models['rf']
    svm_model = best_models['svm']
    mlp_model = best_models['mlp']
    lgb_model = best_models['lgb']

    # 使用 CalibratedClassifierCV 对模型进行校准
    models = [
        CalibratedClassifierCV(xgb_model, cv=5, method='sigmoid'),
        CalibratedClassifierCV(logistic_model, cv=5, method='sigmoid'),
        CalibratedClassifierCV(rf_model, cv=5, method='sigmoid'),
        CalibratedClassifierCV(svm_model, cv=5, method='sigmoid'),
        CalibratedClassifierCV(mlp_model, cv=5, method='sigmoid'),
        CalibratedClassifierCV(lgb_model, cv=5, method='sigmoid')
    ]
    
    # 训练校准后的模型
    for model in models:
        model.fit(X_train, y_train)
    
    return models

# 模型名称
model_names = ['XGBoost', 'LR', 'RF', 'SVM', 'MLP', 'GBM']

# 训练校准后的模型
models = train_and_calibrate_models(best_models, X_train, y_train)

# 绘制校准曲线并计算 Brier 分数，使用外部验证集 (external validation set)
plot_calibration_curves(models, model_names, X_external, y_external, 'Validation Set')

#训练集DCA
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# 函数：计算模型的净效益
def calculate_net_benefit_model(thresh_group, y_pred_score, y_label):
    net_benefit_model = np.zeros_like(thresh_group)
    n = len(y_label)
    for i, thresh in enumerate(thresh_group):
        y_pred_label = y_pred_score > thresh
        tn, fp, fn, tp = confusion_matrix(y_label, y_pred_label).ravel()
        net_benefit = (tp / n) - (fp / n) * (thresh / (1 - thresh))
        net_benefit_model[i] = net_benefit
    return net_benefit_model

# 函数：计算Treat All策略的净效益
def calculate_net_benefit_all(thresh_group, y_label):
    net_benefit_all = np.zeros_like(thresh_group)
    n = len(y_label)
    for i, thresh in enumerate(thresh_group):
        predictions = np.ones(n)  # 假设全体都被预测为阳性
        tn, fp, fn, tp = confusion_matrix(y_label, predictions).ravel()
        net_benefit = (tp / n) - (fp / n) * (thresh / (1 - thresh))
        net_benefit_all[i] = net_benefit
    return net_benefit_all

# 函数：绘制DCA曲线
def plot_DCA(ax, thresh_group, net_benefit_model, label, color):
    ax.plot(thresh_group, net_benefit_model, label=label, color=color, linewidth=2)
    ax.set_xlim(0, 1)
    ax.set_ylim(net_benefit_model.min() - 0.05, net_benefit_model.max() + 0.05)
    ax.set_xlabel('Threshold Probability', fontsize=12, fontweight='bold', fontfamily='Arial')
    ax.set_ylabel('Net Benefit', fontsize=12, fontweight='bold', fontfamily='Arial')
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.legend(loc='upper right', fontsize=10, frameon=False)
    return ax

# 主绘图代码
def plot_decision_curve_analysis(models, model_names, thresh_group, X_train, y_train, fig_name='dca_plot.png'):
    fig, ax = plt.subplots(figsize=(8, 6))

    # 计算Treat All策略的净效益
    net_benefit_all = calculate_net_benefit_all(thresh_group, y_train)

    # Treat all和Treat none的基线策略
    ax.plot(thresh_group, net_benefit_all, color='black', linestyle='-', label='Treat all', linewidth=2)
    ax.plot((0, 1), (0, 0), color='gray', linestyle='--', label='Treat none', linewidth=2)

    # 计算并绘制每个模型的净效益曲线
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
    for i, (label, model) in enumerate(models.items()):
        # 获取每个模型的预测概率
        preds = model.predict_proba(X_train)[:, 1]  # 使用模型的训练集预测概率
        net_benefit_model = calculate_net_benefit_model(thresh_group, preds, y_train)
        # 使用model_names中的对应名称作为图例
        ax = plot_DCA(ax, thresh_group, net_benefit_model, model_names[i], colors[i])

    # 图像的美观设置
    ax.set_title('Decision Curve Analysis (Training set)', fontsize=16, fontweight='bold', fontfamily='Arial', pad=20)
    ax.tick_params(axis='both', which='major', labelsize=12)
    
    # 保存图像
    plt.tight_layout()
    plt.savefig(fig_name, dpi=300, bbox_inches='tight')
    plt.show()

# 测试数据
thresh_group = np.linspace(0.01, 0.99, 100)

# 模型名称
model_names = ['LR', 'RF', 'SVM', 'XGBoost', 'GBM', 'MLP']

# 调用绘制函数，使用最佳模型
plot_decision_curve_analysis(best_models, model_names, thresh_group, X_train, y_train, fig_name='dca_1.png')

#测试集DCA
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# 函数：计算模型的净效益
def calculate_net_benefit_model(thresh_group, y_pred_score, y_label):
    net_benefit_model = np.zeros_like(thresh_group)
    n = len(y_label)
    for i, thresh in enumerate(thresh_group):
        y_pred_label = y_pred_score > thresh
        tn, fp, fn, tp = confusion_matrix(y_label, y_pred_label).ravel()
        net_benefit = (tp / n) - (fp / n) * (thresh / (1 - thresh))
        net_benefit_model[i] = net_benefit
    return net_benefit_model

# 函数：计算Treat All策略的净效益
def calculate_net_benefit_all(thresh_group, y_label):
    net_benefit_all = np.zeros_like(thresh_group)
    n = len(y_label)
    for i, thresh in enumerate(thresh_group):
        predictions = np.ones(n)  # 假设全体都被预测为阳性
        tn, fp, fn, tp = confusion_matrix(y_label, predictions).ravel()
        net_benefit = (tp / n) - (fp / n) * (thresh / (1 - thresh))
        net_benefit_all[i] = net_benefit
    return net_benefit_all

# 函数：绘制DCA曲线
def plot_DCA(ax, thresh_group, net_benefit_model, label, color):
    ax.plot(thresh_group, net_benefit_model, label=label, color=color, linewidth=2)
    ax.set_xlim(0, 1)
    ax.set_ylim(net_benefit_model.min() - 0.05, net_benefit_model.max() + 0.05)
    ax.set_xlabel('Threshold Probability', fontsize=12, fontweight='bold', fontfamily='Arial')
    ax.set_ylabel('Net Benefit', fontsize=12, fontweight='bold', fontfamily='Arial')
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.legend(loc='upper right', fontsize=10, frameon=False)
    return ax

# 主绘图代码
def plot_decision_curve_analysis(models, model_names, thresh_group, X_test, y_test, fig_name='dca_plot.png'):
    fig, ax = plt.subplots(figsize=(8, 6))

    # 计算Treat All策略的净效益
    net_benefit_all = calculate_net_benefit_all(thresh_group, y_test)

    # Treat all和Treat none的基线策略
    ax.plot(thresh_group, net_benefit_all, color='black', linestyle='-', label='Treat all', linewidth=2)
    ax.plot((0, 1), (0, 0), color='gray', linestyle='--', label='Treat none', linewidth=2)

    # 计算并绘制每个模型的净效益曲线
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
    for i, (label, model) in enumerate(models.items()):
        # 获取每个模型的预测概率
        preds = model.predict_proba(X_test)[:, 1]  # 使用模型的预测概率
        net_benefit_model = calculate_net_benefit_model(thresh_group, preds, y_test)
        # 使用model_names中的对应名称作为图例
        ax = plot_DCA(ax, thresh_group, net_benefit_model, model_names[i], colors[i])

    # 图像的美观设置
    ax.set_title('Decision Curve Analysis (Test set)', fontsize=16, fontweight='bold', fontfamily='Arial', pad=20)
    ax.tick_params(axis='both', which='major', labelsize=12)
    
    # 保存图像
    plt.tight_layout()
    plt.savefig(fig_name, dpi=300, bbox_inches='tight')
    plt.show()

# 测试数据
thresh_group = np.linspace(0.01, 0.99, 100)

# 模型名称
model_names = ['LR', 'RF', 'SVM', 'XGBoost', 'GBM', 'MLP']

# 调用绘制函数，使用最佳模型
plot_decision_curve_analysis(best_models, model_names, thresh_group, X_test, y_test, fig_name='dca_2.png')

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# 函数：计算模型的净效益
def calculate_net_benefit_model(thresh_group, y_pred_score, y_label):
    net_benefit_model = np.zeros_like(thresh_group)
    n = len(y_label)
    for i, thresh in enumerate(thresh_group):
        y_pred_label = y_pred_score > thresh
        tn, fp, fn, tp = confusion_matrix(y_label, y_pred_label).ravel()
        net_benefit = (tp / n) - (fp / n) * (thresh / (1 - thresh))
        net_benefit_model[i] = net_benefit
    return net_benefit_model

# 函数：计算Treat All策略的净效益
def calculate_net_benefit_all(thresh_group, y_label):
    net_benefit_all = np.zeros_like(thresh_group)
    n = len(y_label)
    for i, thresh in enumerate(thresh_group):
        predictions = np.ones(n)  # 假设全体都被预测为阳性
        tn, fp, fn, tp = confusion_matrix(y_label, predictions).ravel()
        net_benefit = (tp / n) - (fp / n) * (thresh / (1 - thresh))
        net_benefit_all[i] = net_benefit
    return net_benefit_all

# 函数：绘制DCA曲线
def plot_DCA(ax, thresh_group, net_benefit_model, label, color):
    ax.plot(thresh_group, net_benefit_model, label=label, color=color, linewidth=2)
    ax.set_xlim(0, 1)
    ax.set_ylim(net_benefit_model.min() - 0.05, net_benefit_model.max() + 0.05)
    ax.set_xlabel('Threshold Probability', fontsize=12, fontweight='bold', fontfamily='Arial')
    ax.set_ylabel('Net Benefit', fontsize=12, fontweight='bold', fontfamily='Arial')
    ax.grid(True, linestyle='--', alpha=0.7)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.legend(loc='upper right', fontsize=10, frameon=False)
    return ax

# 主绘图代码
def plot_decision_curve_analysis(models, model_names, thresh_group, X_external, y_external, fig_name='dca_plot.png'):
    fig, ax = plt.subplots(figsize=(8, 6))

    # 计算Treat All策略的净效益
    net_benefit_all = calculate_net_benefit_all(thresh_group, y_external)

    # Treat all和Treat none的基线策略
    ax.plot(thresh_group, net_benefit_all, color='black', linestyle='-', label='Treat all', linewidth=2)
    ax.plot((0, 1), (0, 0), color='gray', linestyle='--', label='Treat none', linewidth=2)

    # 计算并绘制每个模型的净效益曲线
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', '#8c564b']
    for i, (label, model) in enumerate(models.items()):
        # 获取每个模型在外部验证集上的预测概率
        preds = model.predict_proba(X_external)[:, 1]  # 使用模型的预测概率
        net_benefit_model = calculate_net_benefit_model(thresh_group, preds, y_external)
        # 使用model_names中的对应名称作为图例
        ax = plot_DCA(ax, thresh_group, net_benefit_model, model_names[i], colors[i])

    # 图像的美观设置
    ax.set_title('Decision Curve Analysis (Validation Set)', fontsize=16, fontweight='bold', fontfamily='Arial', pad=20)
    ax.tick_params(axis='both', which='major', labelsize=12)
    
    # 保存图像
    plt.tight_layout()
    plt.savefig(fig_name, dpi=300, bbox_inches='tight')
    plt.show()

# 使用外部验证集进行DCA分析
thresh_group = np.linspace(0.01, 0.99, 100)

# 模型名称
model_names = ['LR', 'RF','SVM','XGBoost',   'GBM' ,'MLP']

# 调用绘制函数，使用最佳模型
plot_decision_curve_analysis(best_models, model_names, thresh_group, X_external, y_external, fig_name='dca_3.png')

#测试集PR曲线
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import precision_recall_curve, auc
from sklearn.utils import resample

def plot_pr_curve(ax, y_true, y_scores, label, color, n_bootstraps=1000, alpha=0.95):
    """绘制PR曲线并计算AUC及95%置信区间"""
    # 将y_true和y_scores转换为NumPy数组，避免pandas的索引问题
    y_true = np.array(y_true)
    y_scores = np.array(y_scores)
    
    # 计算 PR 曲线
    precision, recall, _ = precision_recall_curve(y_true, y_scores)
    auc_score = auc(recall, precision)

    # Bootstrap 计算 95% 置信区间
    bootstrapped_scores = []
    rng = np.random.RandomState(seed=42)  # 设置随机种子确保可复现性
    for _ in range(n_bootstraps):
        indices = resample(np.arange(len(y_scores)), random_state=rng)
        if len(np.unique(y_true[indices])) < 2:
            continue
        precision_boot, recall_boot, _ = precision_recall_curve(y_true[indices], y_scores[indices])
        auc_boot = auc(recall_boot, precision_boot)
        bootstrapped_scores.append(auc_boot)
    
    # 计算置信区间
    sorted_scores = np.array(bootstrapped_scores)
    lower_bound = np.percentile(sorted_scores, ((1 - alpha) / 2) * 100)
    upper_bound = np.percentile(sorted_scores, (alpha + (1 - alpha) / 2) * 100)
    
    # 绘制 PR 曲线并在图例中显示 AUC 和 95% 置信区间
    ax.plot(recall, precision, 
            label=f'{label} (AUC = {auc_score:.3f} 95% CI [{lower_bound:.3f}-{upper_bound:.3f}])', 
            color=color, linewidth=2)
    
    # 设置图表样式
    ax.set_xlabel("Recall", fontdict={'family': 'Times New Roman', 'fontsize': 15})
    ax.set_ylabel("Precision", fontdict={'family': 'Times New Roman', 'fontsize': 15})

    ax.legend(loc='lower left', fontsize=12)
    return ax


# 绘制测试集的PR曲线
fig_test, ax_test = plt.subplots(figsize=(8, 6))

# 自定义颜色列表
colors = plt.get_cmap("tab10").colors

# 使用最佳模型进行预测并绘制PR曲线
# SVM模型
svm_preds_test = best_models['svm'].predict_proba(X_test)[:, 1]
ax_test = plot_pr_curve(ax_test, y_test, svm_preds_test, 'SVM', colors[0])

# XGBoost模型
xgb_preds_test = best_models['xgb'].predict_proba(X_test)[:, 1]
ax_test = plot_pr_curve(ax_test, y_test, xgb_preds_test, 'XGBoost', colors[1])

# GBM模型
gbm_preds_test = best_models['lgb'].predict_proba(X_test)[:, 1]
ax_test = plot_pr_curve(ax_test, y_test, gbm_preds_test, 'GBM', colors[2])

# 逻辑回归模型
log_reg_preds_test = best_models['log_reg'].predict_proba(X_test)[:, 1]
ax_test = plot_pr_curve(ax_test, y_test, log_reg_preds_test, 'LR', colors[3])

# 随机森林模型
rf_preds_test = best_models['rf'].predict_proba(X_test)[:, 1]
ax_test = plot_pr_curve(ax_test, y_test, rf_preds_test, 'RF', colors[4])

# MLP模型
mlp_preds_test = best_models['mlp'].predict_proba(X_test)[:, 1]
ax_test = plot_pr_curve(ax_test, y_test, mlp_preds_test, 'MLP', colors[5])

# 设置图标题和显示
plt.title('Precision-Recall Curves (Test set)', fontdict={'family': 'Times New Roman', 'fontsize': 18, 'fontweight': 'bold'})
plt.tight_layout()
plt.savefig("pr_curves_1.png")
plt.show()

#各模型训练集及测试集的预测指标
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score, roc_auc_score

# 计算指标函数
def calculate_metrics(y_true, y_pred, y_scores):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # 防止分母为零
    f1 = f1_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_scores)  # 计算AUC
    return accuracy, sensitivity, precision, specificity, f1, auc

# 打印模型评估指标函数
def print_model_metrics(model, X_train, y_train, X_test, y_test, model_name):
    if hasattr(model, "predict_proba"):
        y_train_scores = model.predict_proba(X_train)[:, 1]  # 用 predict_proba 得到分数
        y_test_scores = model.predict_proba(X_test)[:, 1]
        y_train_pred = (y_train_scores > 0.5).astype(int)
        y_test_pred = (y_test_scores > 0.5).astype(int)
    else:
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        y_train_scores = y_train_pred  # 对于不提供概率的模型，使用预测值作为近似分数
        y_test_scores = y_test_pred

    # 计算训练集和测试集的评估指标
    accuracy_train, sensitivity_train, precision_train, specificity_train, f1_train, auc_train = calculate_metrics(y_train, y_train_pred, y_train_scores)
    accuracy_test, sensitivity_test, precision_test, specificity_test, f1_test, auc_test = calculate_metrics(y_test, y_test_pred, y_test_scores)

    # 输出训练集和测试集的结果
    print(f"--- {model_name} ---")
    print(f"Training Set Metrics:")
    print(f"Accuracy: {accuracy_train:.4f}, Sensitivity: {sensitivity_train:.4f}, Precision: {precision_train:.4f}, Specificity: {specificity_train:.4f}, F1 Score: {f1_train:.4f}, AUC: {auc_train:.4f}")
    
    print(f"Test Set Metrics:")
    print(f"Accuracy: {accuracy_test:.4f}, Sensitivity: {sensitivity_test:.4f}, Precision: {precision_test:.4f}, Specificity: {specificity_test:.4f}, F1 Score: {f1_test:.4f}, AUC: {auc_test:.4f}")
    print()

# 使用之前找到的最佳模型
models = {
    'SVM': best_models['svm'],
    'XGBoost': best_models['xgb'],
    'LightGBM': best_models['lgb'],
    'Logistic Regression': best_models['log_reg'],
    'Random Forest': best_models['rf'],
    'MLP': best_models['mlp']
}

# 假设你已经有 X_train, y_train, X_test, y_test 数据集
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)  # 确保每个模型都被训练
    print_model_metrics(model, X_train, y_train, X_test, y_test, model_name)

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import xgboost as xgb
import shap
import matplotlib.pyplot as plt

# 加载数据
data = pd.read_excel('1.xlsx')

# 筛选特征
selected_features = ['adlab_c', 'systo', 'diasto', 'bmi', 'bl_crea', 'bl_cho', 'bl_tg', 'bl_hdl', 'bl_ldl', 'bl_ua', 'bl_hct', 'bl_hgb', 'bl_cysc', 'age', 'iadl', 'tyg', 'grip']

X = data[selected_features]
y = data['diagnosis']

# 分割数据集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=123)

# 创建并训练XGBoost模型，使用指定参数
xgb_model = xgb.XGBClassifier(learning_rate: 0.1, max_depth: 3, n_estimators: 100, subsample: 0.8)
xgb_model.fit(X_train, y_train)

# 使用TreeExplainer来计算SHAP值
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test)

# 创建SHAP蜂群图（dot plot）
shap.summary_plot(shap_values, X_test, plot_type="dot")

# 创建变量重要性排序图（bar plot）
shap.summary_plot(shap_values, X_test, plot_type="bar")

# 创建SHAP散点图（以'ADL score'为例）
shap.dependence_plot("age", shap_values, X_test)

# 选择第 1 类的 SHAP 值
shap_values_class_1 = shap_values[:, 0]
